In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample poem data
poems = [
    "The rose is red",
    "The violet's blue",
    "Sugar is sweet",
    "And so are you"
]

# Tokenize words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(poems)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in poems:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Create predictors and label
X, y = input_sequences[:,:-1],input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, epochs=100, verbose=1)

# Function to generate text
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = tf.argmax(predicted_probs, axis=-1).numpy()
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Generate poem based on user prompts
seed_text = input("Enter a prompt: ")
num_words = int(input("Enter number of words for the poem: "))
generated_poem = generate_text(seed_text, num_words, max_sequence_len)
print(generated_poem)


Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 2.5643 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 16ms/step - loss: 2.5550 - accuracy: 0.3000
Epoch 3/100
1/1 [==============================] - 0s 15ms/step - loss: 2.5457 - accuracy: 0.4000
Epoch 4/100
1/1 [==============================] - 0s 14ms/step - loss: 2.5363 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 17ms/step - loss: 2.5268 - accuracy: 0.6000
Epoch 6/100
1/1 [==============================] - 0s 17ms/step - loss: 2.5169 - accuracy: 0.6000
Epoch 7/100
1/1 [==============================] - 0s 15ms/step - loss: 2.5065 - accuracy: 0.6000
Epoch 8/100
1/1 [==============================] - 0s 16ms/step - loss: 2.4956 - accuracy: 0.6000
Epoch 9/100
1/1 [==============================] - 0s 15ms/step - loss: 2.4840 - accuracy: 0.6000
Epoch 10/100
1/1 [==============================] - 0s 15ms/step - loss: 2.4717 - accuracy: 0.6000
Epoch 11/100
1/1 

In [3]:
import numpy as np

# Function to generate text with randomness
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        # Introduce randomness by sampling from the predicted probabilities
        predicted_index = np.random.choice(len(predicted_probs), p=predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Generate poem based on user prompts
seed_text = input("Enter a prompt: ")
num_words = int(input("Enter number of words for the poem: "))
generated_poem = generate_text(seed_text, num_words, max_sequence_len)
print(generated_poem)


Enter a prompt: sad
Enter number of words for the poem: 24
sad so rose are you red is red you the rose is red red red red sugar violet's red red are you sweet you you


In [4]:
# Function to generate text with temperature-based sampling
def generate_text(seed_text, next_words, max_sequence_len, temperature=1.0):
    generated_text = seed_text
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        # Apply temperature to the predicted probabilities
        predicted_probs = np.log(predicted_probs) / temperature
        exp_preds = np.exp(predicted_probs)
        predicted_probs = exp_preds / np.sum(exp_preds)
        # Sample from the adjusted probabilities
        predicted_index = np.random.choice(len(predicted_probs), p=predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
        generated_text += " " + output_word
    return generated_text

# Generate poem based on user prompts with temperature-based sampling
seed_text = input("Enter a prompt: ")
num_words = int(input("Enter number of words for the poem: "))
temperature = float(input("Enter temperature for sampling (e.g., 0.5 for lower randomness, 1.0 for default, 1.5 for higher randomness): "))
generated_poem = generate_text(seed_text, num_words, max_sequence_len, temperature)
print(generated_poem)


Enter a prompt: sad
Enter number of words for the poem: 24
Enter temperature for sampling (e.g., 0.5 for lower randomness, 1.0 for default, 1.5 for higher randomness): 0.5
sad rose is sweet red you are you you you blue red red is red red  you you sweet sweet red you you sweet


In [5]:
pip install transformers


In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Download GPT-2 tokenizer and model files
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Save tokenizer and model to a local directory
tokenizer.save_pretrained("./gpt2-medium")
model.save_pretrained("./gpt2-medium")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer from the local directory
model_name = "./gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Generate poem based on user prompt
seed_text = input("Enter a prompt: ")
num_words = int(input("Enter number of words for the poem: "))

# Tokenize input text
input_ids = tokenizer.encode(seed_text, return_tensors="pt")

# Generate poem
output = model.generate(input_ids, max_length=num_words, num_return_sequences=1, temperature=0.7)
generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_poem)


Enter a prompt: sad
Enter number of words for the poem: 24


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


sadness, and the desire to be loved.

The first step in the process of healing is to recognize


In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer from the local directory
model_name = "./gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Generate poem based on user prompt
seed_text = input("Enter a prompt: ")
num_words = int(input("Enter number of words for the poem: "))

# Tune the temperature parameter
temperature = 0.3  # Adjust temperature to control randomness

# Adjust the length of the generated text
num_return_sequences = 1  # Generate only one poem

# Tokenize input text
input_ids = tokenizer.encode(seed_text, return_tensors="pt")

# Generate poem
output = model.generate(
    input_ids,
    max_length=num_words,
    num_return_sequences=num_return_sequences,
    temperature=temperature,
    do_sample=True  # Enable sampling
)

# Decode the generated poem
generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated poem
print(generated_poem)


Enter a prompt: happy
Enter number of words for the poem: 24


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


happy, but I'm not sure how much of that is due to the fact that I'm a woman.




In [14]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer from the local directory
model_name = "./gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Generate poem based on user prompt
seed_text = input("Enter a prompt: ")
num_lines = int(input("Enter number of lines for the poem: "))

# Tune the temperature parameter
temperature = 0.3  # Adjust temperature to control randomness

# Adjust the length of the generated text
num_words_per_line = 6  # Adjust the number of words per line
total_num_words = num_lines * num_words_per_line

# Tokenize input text
input_ids = tokenizer.encode(seed_text, return_tensors="pt")

# Generate poem
output = model.generate(
    input_ids,
    max_length=total_num_words,
    num_return_sequences=1,
    temperature=temperature,
    do_sample=True  # Enable sampling
)

# Decode the generated poem
generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)

# Insert line breaks to format the poem
lines = [generated_poem[i:i+num_words_per_line*5] for i in range(0, len(generated_poem), num_words_per_line*5)]
formatted_poem = "\n".join(lines)

# Print the formatted poem
print(formatted_poem)


Enter a prompt: sad
Enter number of lines for the poem: 4


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


sadness and sadness, and the d
esire to be a part of somethin
g bigger than themselves.

I t
hink that


In [15]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re

# Load pre-trained GPT-2 model and tokenizer from the local directory
model_name = "./gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Generate poem based on user prompt
seed_text = input("Enter a prompt: ")
num_lines = int(input("Enter number of lines for the poem: "))

# Tune the temperature parameter
temperature = 0.3  # Adjust temperature to control randomness

# Adjust the length of the generated text
num_words_per_line = 6  # Adjust the number of words per line
total_num_words = num_lines * num_words_per_line

# Tokenize input text
input_ids = tokenizer.encode(seed_text, return_tensors="pt")

# Generate poem
output = model.generate(
    input_ids,
    max_length=total_num_words,
    num_return_sequences=1,
    temperature=temperature,
    do_sample=True  # Enable sampling
)

# Decode the generated poem
generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)

# Split the generated poem into lines based on punctuation marks
lines = re.split(r'[.,;]', generated_poem)
formatted_poem = "\n".join(lines[:num_lines])  # Select the desired number of lines

# Print the formatted poem
print(formatted_poem)


Enter a prompt: sad
Enter number of lines for the poem: 4


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


sad
 but it's not a bad thing


"The best thing you can do is to be patient


In [19]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re
import torch

# Load pre-trained GPT-2 model and tokenizer from the local directory
model_name = "./gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token to EOS token
tokenizer.pad_token = tokenizer.eos_token

# Initialize an empty dataset
dataset = []

# Define fine-tuning parameters
learning_rate = 5e-5
num_epochs = 3  # Adjust as needed

# Continuous generation loop
while True:
    # Prompt user for input
    seed_text = input("Enter a prompt (or type 'exit' to stop): ")
    if seed_text.lower() == "exit":
        break

    # Generate poem based on user prompt
    num_lines = 4  # Adjust the number of lines for the poem
    temperature = 0.3  # Adjust temperature to control randomness
    num_words_per_line = 6  # Adjust the number of words per line
    total_num_words = num_lines * num_words_per_line

    input_ids = tokenizer.encode(seed_text, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=total_num_words,
        num_return_sequences=1,
        temperature=temperature,
        do_sample=True
    )
    generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)

    # Print the generated poem
    print("Generated Poem:")
    print(generated_poem)

    # Add the generated poem to the dataset
    dataset.append(generated_poem)

    # Fine-tune the model with the updated dataset
    print("Fine-tuning the model with the updated dataset...")
    for epoch in range(num_epochs):
        tokenized_dataset = tokenizer(dataset, return_tensors="pt", truncation=True, padding=True)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        model.train()
        optimizer.zero_grad()
        outputs = model(**tokenized_dataset, labels=tokenized_dataset["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}: Loss = {loss.item()}")

print("Program terminated.")


Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sad.

"We're not going to be able to do that," he said. "We're not
Fine-tuning the model with the updated dataset...
Epoch 1/3: Loss = 1.6225327253341675
Epoch 2/3: Loss = 0.8021121621131897
Epoch 3/3: Loss = 0.35355013608932495
Enter a prompt (or type 'exit' to stop): happy


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
happyadventure.com/

Twitter: @adam_barnes

Admission: $10
Fine-tuning the model with the updated dataset...
Epoch 1/3: Loss = 0.856410801410675
Epoch 2/3: Loss = 0.33163365721702576
Epoch 3/3: Loss = 0.22955183684825897
Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sad.

"We're not going to be able to do that," he said. "We're not
Fine-tuning the model with the updated dataset...
Epoch 1/3: Loss = 0.2481382042169571
Epoch 2/3: Loss = 0.07008777558803558
Epoch 3/3: Loss = 0.05267786607146263
Enter a prompt (or type 'exit' to stop): exit
Program terminated.


In [20]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained GPT-2 model and tokenizer from the local directory
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token to EOS token
tokenizer.pad_token = tokenizer.eos_token

# Initialize an empty dataset
dataset = []

# Define fine-tuning parameters
learning_rate = 5e-5
num_epochs = 5  # Extend the number of fine-tuning epochs

# Continuous generation loop
while True:
    # Prompt user for input
    seed_text = input("Enter a prompt (or type 'exit' to stop): ")
    if seed_text.lower() == "exit":
        break

    # Generate poem based on user prompt
    num_lines = 4  # Adjust the number of lines for the poem
    temperature = 0.7  # Adjust temperature to control randomness
    num_words_per_line = 6  # Adjust the number of words per line
    total_num_words = num_lines * num_words_per_line

    input_ids = tokenizer.encode(seed_text, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=total_num_words,
        num_return_sequences=1,
        temperature=temperature,
        do_sample=True
    )
    generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)

    # Print the generated poem
    print("Generated Poem:")
    print(generated_poem)

    # Add the generated poem to the dataset
    dataset.append(generated_poem)

    # Fine-tune the model with the updated dataset
    print("Fine-tuning the model with the updated dataset...")
    for epoch in range(num_epochs):
        tokenized_dataset = tokenizer(dataset, return_tensors="pt", truncation=True, padding=True)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        model.train()
        optimizer.zero_grad()
        outputs = model(**tokenized_dataset, labels=tokenized_dataset["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}: Loss = {loss.item()}")

print("Program terminated.")


Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sad to see the whole country turning on her and thinking, 'We won't be able to protect our children,'"
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 3.562469482421875
Epoch 2/5: Loss = 2.0597400665283203
Epoch 3/5: Loss = 1.052342414855957
Epoch 4/5: Loss = 0.4707696735858917
Epoch 5/5: Loss = 0.08873549848794937
Enter a prompt (or type 'exit' to stop): happy


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
happyad"

"The whole country is in a state of shock and a sense of loss," said Anand
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 0.8382892608642578
Epoch 2/5: Loss = 0.5255614519119263
Epoch 3/5: Loss = 0.22400596737861633
Epoch 4/5: Loss = 0.10175915062427521
Epoch 5/5: Loss = 0.11186199635267258
Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sad to see the whole country turning on her and thinking, 'We won't be able to protect our children,'"
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 0.04418128356337547
Epoch 2/5: Loss = 0.0643690899014473
Epoch 3/5: Loss = 0.0023731158580631018
Epoch 4/5: Loss = 0.1977422535419464
Epoch 5/5: Loss = 0.0872376412153244
Enter a prompt (or type 'exit' to stop): exit
Program terminated.


In [21]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained GPT-2 model and tokenizer from the local directory
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token to EOS token
tokenizer.pad_token = tokenizer.eos_token

# Initialize an empty dataset
dataset = []

# Define fine-tuning parameters
learning_rate = 5e-5
num_epochs = 5  # Extend the number of fine-tuning epochs

# Continuous generation loop
while True:
    # Prompt user for input
    seed_text = input("Enter a prompt (or type 'exit' to stop): ")
    if seed_text.lower() == "exit":
        break

    # Generate poem based on user prompt
    num_lines = 4  # Adjust the number of lines for the poem
    temperature = 0.9  # Adjust temperature to control randomness
    num_words_per_line = 6  # Adjust the number of words per line
    total_num_words = num_lines * num_words_per_line

    input_ids = tokenizer.encode(seed_text, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=total_num_words,
        num_return_sequences=1,
        temperature=temperature,
        do_sample=True
    )
    generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)

    # Print the generated poem
    print("Generated Poem:")
    print(generated_poem)

    # Add the generated poem to the dataset
    dataset.append(generated_poem)

    # Fine-tune the model with the updated dataset
    print("Fine-tuning the model with the updated dataset...")
    for epoch in range(num_epochs):
        tokenized_dataset = tokenizer(dataset, return_tensors="pt", truncation=True, padding=True)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        model.train()
        optimizer.zero_grad()
        outputs = model(**tokenized_dataset, labels=tokenized_dataset["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}: Loss = {loss.item()}")

print("Program terminated.")


Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sad, but he also said he's hopeful about the direction of the school and he looks forward to having a full
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 3.4879868030548096
Epoch 2/5: Loss = 2.0759036540985107
Epoch 3/5: Loss = 1.1668250560760498
Epoch 4/5: Loss = 0.5403873324394226
Epoch 5/5: Loss = 0.2648612856864929
Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sad, but he also feels it's more in the right direction. He's looking forward to having a full season
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 0.9908506870269775
Epoch 2/5: Loss = 0.492975115776062
Epoch 3/5: Loss = 0.3094403147697449
Epoch 4/5: Loss = 0.23198886215686798
Epoch 5/5: Loss = 0.19905872642993927
Enter a prompt (or type 'exit' to stop): happy


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
happyad, of course, but he also said he's looking forward to having a full season in 2016 after having a
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 0.5013126134872437
Epoch 2/5: Loss = 0.42757290601730347
Epoch 3/5: Loss = 0.19930203258991241
Epoch 4/5: Loss = 0.1361730694770813
Epoch 5/5: Loss = 0.15019449591636658
Enter a prompt (or type 'exit' to stop): exit
Program terminated.


In [22]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained GPT-2 model and tokenizer from the local directory
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token to EOS token
tokenizer.pad_token = tokenizer.eos_token

# Initialize an empty dataset
dataset = []

# Define fine-tuning parameters
learning_rate = 5e-5
num_epochs = 5  # Extend the number of fine-tuning epochs

# Continuous generation loop
while True:
    # Prompt user for input
    seed_text = input("Enter a prompt (or type 'exit' to stop): ")
    if seed_text.lower() == "exit":
        break

    # Generate poem based on user prompt
    num_lines = 6  # Extend the number of lines for the poem
    temperature = 0.7  # Adjust temperature to control randomness
    num_words_per_line = 8  # Adjust the number of words per line
    total_num_words = num_lines * num_words_per_line

    input_ids = tokenizer.encode(seed_text, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=total_num_words,
        num_return_sequences=1,
        temperature=temperature,
        do_sample=True
    )
    generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)

    # Print the generated poem
    print("Generated Poem:")
    print(generated_poem)

    # Add the generated poem to the dataset
    dataset.append(generated_poem)

    # Fine-tune the model with the updated dataset
    print("Fine-tuning the model with the updated dataset...")
    for epoch in range(num_epochs):
        tokenized_dataset = tokenizer(dataset, return_tensors="pt", truncation=True, padding=True)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        model.train()
        optimizer.zero_grad()
        outputs = model(**tokenized_dataset, labels=tokenized_dataset["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}: Loss = {loss.item()}")

print("Program terminated.")


Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sad?

I have seen an awful lot of people who have had success with this technique, some of which I have personally seen work as well. For example, I have seen quite a few people successfully use it on their very
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 2.4382739067077637
Epoch 2/5: Loss = 1.7234443426132202
Epoch 3/5: Loss = 1.0249850749969482
Epoch 4/5: Loss = 0.59031742811203
Epoch 5/5: Loss = 0.36385130882263184
Enter a prompt (or type 'exit' to stop): happy


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
happyadam@gmail.com

You can also email me at:

jimmyers@gmail.com

Thanks for reading.

Yours in the field,

Jim

P.S
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 0.8545478582382202
Epoch 2/5: Loss = 0.5677691102027893
Epoch 3/5: Loss = 0.3135366439819336
Epoch 4/5: Loss = 0.3480299413204193
Epoch 5/5: Loss = 0.5190442800521851
Enter a prompt (or type 'exit' to stop): exit
Program terminated.


In [23]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained GPT-2 model and tokenizer from the local directory
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token to EOS token
tokenizer.pad_token = tokenizer.eos_token

# Initialize an empty dataset
dataset = []

# Define fine-tuning parameters
learning_rate = 5e-5
num_epochs = 5  # Extend the number of fine-tuning epochs

# Continuous generation loop
while True:
    # Prompt user for input
    seed_text = input("Enter a prompt (or type 'exit' to stop): ")
    if seed_text.lower() == "exit":
        break

    # Generate poem based on user prompt
    num_lines = 4  # Adjust the number of lines for the poem
    temperature = 0.7  # Adjust temperature to control randomness
    num_words_per_line = 6  # Adjust the number of words per line
    total_num_words = num_lines * num_words_per_line

    input_ids = tokenizer.encode(seed_text, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=total_num_words,
        num_return_sequences=1,
        temperature=temperature,
        do_sample=True
    )
    generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)

    # Format the poem into separate lines
    lines = generated_poem.split(".")  # Split into lines based on period
    formatted_poem = "\n".join(lines)

    # Print the formatted poem
    print("Generated Poem:")
    print(formatted_poem)

    # Add the generated poem to the dataset
    dataset.append(generated_poem)

    # Fine-tune the model with the updated dataset
    print("Fine-tuning the model with the updated dataset...")
    for epoch in range(num_epochs):
        tokenized_dataset = tokenizer(dataset, return_tensors="pt", truncation=True, padding=True)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        model.train()
        optimizer.zero_grad()
        outputs = model(**tokenized_dataset, labels=tokenized_dataset["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}: Loss = {loss.item()}")

print("Program terminated.")


Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sadness of the poor people who are left behind," and "that they need a voice in the political process"
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 3.158965826034546
Epoch 2/5: Loss = 1.747108817100525
Epoch 3/5: Loss = 0.9614275097846985
Epoch 4/5: Loss = 0.40732163190841675
Epoch 5/5: Loss = 0.16416844725608826
Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sadness of the poor people who are left behind," and "that they need a voice in the political process"
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 0.18558751046657562
Epoch 2/5: Loss = 0.004953719675540924
Epoch 3/5: Loss = 0.026355698704719543
Epoch 4/5: Loss = 0.09574460238218307
Epoch 5/5: Loss = 0.00019790383521467447
Enter a prompt (or type 'exit' to stop): extreme sadness


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
extreme sadness of the poor people who are left behind," and "that they need a voice in the political process" are
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 0.6101548075675964
Epoch 2/5: Loss = 0.25321340560913086
Epoch 3/5: Loss = 0.334208220243454
Epoch 4/5: Loss = 0.1795172393321991
Epoch 5/5: Loss = 0.02136942185461521
Enter a prompt (or type 'exit' to stop): exit
Program terminated.


In [24]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import random

# Load pre-trained GPT-2 model and tokenizer from the local directory
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token to EOS token
tokenizer.pad_token = tokenizer.eos_token

# Initialize an empty dataset
dataset = []

# Define fine-tuning parameters
learning_rate = 5e-5
num_epochs = 5  # Extend the number of fine-tuning epochs

# Continuous generation loop
while True:
    # Prompt user for input
    seed_text = input("Enter a prompt (or type 'exit' to stop): ")
    if seed_text.lower() == "exit":
        break

    # Generate poem based on user prompt
    num_lines = 4  # Adjust the number of lines for the poem
    temperature = 0.7  # Adjust temperature to control randomness
    num_words_per_line = 6  # Adjust the number of words per line
    total_num_words = num_lines * num_words_per_line

    input_ids = tokenizer.encode(seed_text, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=total_num_words,
        num_return_sequences=1,
        temperature=temperature,
        do_sample=True
    )
    generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)

    # Format the poem into separate lines and add rhyming words
    lines = generated_poem.split(".")  # Split into lines based on period
    formatted_lines = []
    for i, line in enumerate(lines):
        if i % 2 == 0:  # Ensure rhyming words for alternate lines
            formatted_lines.append(line.strip() + ",")
        else:
            formatted_lines.append(line.strip())
    formatted_poem = "\n".join(formatted_lines)

    # Print the formatted poem
    print("Generated Poem:")
    print(formatted_poem)

    # Add the generated poem to the dataset
    dataset.append(generated_poem)

    # Shuffle the dataset to encourage diversity in training data
    random.shuffle(dataset)

    # Fine-tune the model with the updated dataset
    print("Fine-tuning the model with the updated dataset...")
    for epoch in range(num_epochs):
        tokenized_dataset = tokenizer(dataset, return_tensors="pt", truncation=True, padding=True)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        model.train()
        optimizer.zero_grad()
        outputs = model(**tokenized_dataset, labels=tokenized_dataset["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}: Loss = {loss.item()}")

print("Program terminated.")


Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sad! He's pretty,
He looks like he could break you in half
But there's nothing he's going,
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 3.2324745655059814
Epoch 2/5: Loss = 2.0010416507720947
Epoch 3/5: Loss = 1.1499598026275635
Epoch 4/5: Loss = 0.5869240164756775
Epoch 5/5: Loss = 0.5752580761909485
Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sad! She's pretty,
She has a pretty face
She looks pretty,
She looks like she could break you
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 0.5795656442642212
Epoch 2/5: Loss = 0.3055581748485565
Epoch 3/5: Loss = 0.6791040897369385
Epoch 4/5: Loss = 0.18627607822418213
Epoch 5/5: Loss = 0.44915154576301575
Enter a prompt (or type 'exit' to stop): happy


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
happyad,
After hearing that, he's pretty sure he's pretty sure he's going to get shot
But he,
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 0.8611461520195007
Epoch 2/5: Loss = 0.4817735552787781
Epoch 3/5: Loss = 0.3347002863883972
Epoch 4/5: Loss = 0.33019450306892395
Epoch 5/5: Loss = 0.18746519088745117
Enter a prompt (or type 'exit' to stop): sad


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Poem:
sad! She's pretty,
She looks like she could break you in half
But there's nothing she's going,
Fine-tuning the model with the updated dataset...
Epoch 1/5: Loss = 0.16804826259613037
Epoch 2/5: Loss = 0.25018954277038574
Epoch 3/5: Loss = 0.15559899806976318
Epoch 4/5: Loss = 0.14784382283687592
Epoch 5/5: Loss = 0.0871877446770668
Enter a prompt (or type 'exit' to stop): exit
Program terminated.
